# GLOW  - Content Quality & Details

# Table of Contents  <a class="anchor" id="toc"></a>

* [Content](#top)
    1. Identify and filter out redirects
    [num editors/article](#nea)
    2. translation
    3. [pagelen & pagelen relative (composition)](#pagelen)
    4. wikidata item
    5. article creation date
    6. [pageviews(use/utility)](#pv)
    7. filter for new articles
    8. edits & editors per article
    9. edits & timestamps
    10. editors per article
    11. [talk page activity](#tpa) 
    12. article watch count
    13. [revert rate(use/utility)](#rr)
    14. [links(importance/integration)](#ol)   

In [46]:
# import wmfdata as wmf
import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all

import requests
import re
from urllib import request
import json

import logging
import gc
import weakref
from functools import reduce
from pathlib import Path
import requests
import pprint

import jupyter_contrib_nbextensions
import pandas as pd
import numpy as np

import time
import datetime as dt 
from datetime import datetime, timedelta, date
import dateutil

%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [47]:
%run P1G_1_data_handling.ipynb
%store -r pa_articles_PT18
%store -r IN_median_vi
#%run ./data_collection/collecting_articles.ipynb

Stored 'query_vars' (dict)


# 3. Content Quality<a class="anchor" id="stage1"></a>
[Back to Table of Contents](#toc)

## Get Data

### read files and set variables which will be used in the notebook for querying

In [48]:
#replace wcode & article list for each wiki // AND remember to rename the filename at the very bottom for each wiki
wcode = 'pa'  
article_list = pa_articles_PT18 # "ਲਿੰਡਸੇ_ਵੋਨ", "ਇੰਗਮੇਰ_ਸਟੈਨਮਾਰਕ", "ਬਸਟਰ_ਕੀਟਨ", "ਮਨੋਰੰਜਨ"...

#replace contest start/end date, and country for each GLOW project
contest_start = '2019-10-10'
contest_end = '2020-01-11'
wiki = wcode+'wiki'
quality_vars = dict(
    raw_articles  = article_list, # fyi, this later turns into clean_page_ids > clean_new_page_ids
    project       = wcode+'.wikipedia',
    country_code  = "IN",
    wiki_db       = wiki,
    contest_start = contest_start,
    contest_end   = contest_end,
    contest_start_dt        = datetime.strptime(contest_start, '%Y-%m-%d'),
    contest_end_dt          = datetime.strptime(contest_end, '%Y-%m-%d'),
    contest_end_dt_month    = datetime.strptime(contest_end, '%Y-%m-%d').strftime('%m'),
    contest_end_dt_day      = datetime.strptime(contest_end, '%Y-%m-%d').strftime('%d'),
    contest_end_dt_1M_month = (datetime.strptime(contest_end, '%Y-%m-%d') + timedelta(days=30)).strftime('%m'),
    contest_end_dt_1M_day   = (datetime.strptime(contest_end, '%Y-%m-%d') + timedelta(days=30)).strftime('%d'),
    MWH_SNAPSHOT = last_month.strftime("%Y-%m"),
)

### Get clean article list: id, wiki, redirects <a class="anchor" id="get_clean_list"></a>
[Back to Table of Contents](#toc)

In [49]:
# adapted from https://github.com/nettrom/suggestbot/blob/master/tool-labs/link-rec/link-recommender.py#L208
#https://www.mediawiki.org/wiki/Manual:Redirect_table
#https://www.mediawiki.org/wiki/Manual:Page_table
#https://www.mediawiki.org/wiki/Manual:Pagelinks_table
#--rd.redirect_id -- where is this field located? in which table can it be found?
       
redirects_r = wmf.mariadb.run("""
SELECT 
       p1.page_id  AS page_id,
       p1.page_title AS page_title,
       p1.page_is_redirect AS p1_is_redirect,
       p1.page_len AS page_len,
       p2.page_id AS rpage_id,
       p2.page_title AS rpage_title,
       p2.page_len rpage_len,
       p2.page_is_redirect AS is_double_redirect
FROM page AS p1 
LEFT JOIN redirect AS rd 
    ON p1.page_id=rd.rd_from 
LEFT JOIN page AS p2 
    ON (rd_namespace = p2.page_namespace)
        AND rd_title = p2.page_title  
WHERE p1.page_namespace = 0
      AND p1.page_title IN ({raw_articles})
""".format(**quality_vars), wiki)


#""".format(start="2017-06", end="2018-06"), "wikishared")
#""".format(pa_articles_2018=pa_articles_2018)
# MIN(p1.page_touched) AS last_modified,

In [50]:
# |
#check to see if any of the page_ids are redirects or double redirects
((redirects_r['p1_is_redirect']==1) & (redirects_r['is_double_redirect']==1)).any()

False

In [51]:
# |
#check to see if any of the page_ids are redirects or double redirects
((redirects_r['p1_is_redirect']==1) | (redirects_r['is_double_redirect']==1)).any()

True

In [ ]:
#we do not want any duplicates here
redirects_r[redirects_r.index.duplicated()]

In [52]:
# act on the results from redirects_r

#create a df 
all_surviving_articles = redirects_r[['page_id','page_title', 'page_len']] 

#seperate the redirected items into their own df
redirects = redirects_r.loc[redirects_r['p1_is_redirect']==1]
#pull only p1.page_id, p1.page_title, p1.page_len 
redirect_df = redirects[['page_id','page_title','page_len']] 

#remove the redirect items from the all_surviving_articles df & create global articles df
articles =  all_surviving_articles[~all_surviving_articles.isin(redirect_df)].dropna(how='all')

#create a new wikicode column using quality_vars['wiki_db']
#ffill could also work here
articles['wikicode'] = quality_vars['wiki_db']

In [53]:
# create a tuple of clean non redirect page_ids to query 
clean_pageids = tuple(list(articles['page_id']))
clean_pagetitles = (articles['page_title'])

#non-normalized titles, use spaces instead of underscores and may include namespace name
#create a tuple of clean page_titles denormalized (with spaces instead of underscores) for pulling wiki data items (see below)
clean_titles_denormalized = tuple(list(clean_pagetitles.replace('_', ' ', regex=True)))

#update the query variable to use it in queries
quality_vars.update({'clean_pageids': clean_pageids,
                     'clean_titles_denormalized': clean_titles_denormalized})

## Translation

In [63]:
#https://phabricator.wikimedia.org/T201539

#get list of articles that have an edit associated with the content translation tool
at_edits = hive.run("""
SELECT
    page_id, 
    revision_tags AS at_edits
FROM wmf.mediawiki_history
WHERE
    snapshot = "{MWH_SNAPSHOT}"
    AND event_timestamp >="{contest_start}"
    AND event_timestamp <"{contest_end}"
    AND page_namespace = 0
    AND event_entity = 'revision'
    AND revision_is_identity_reverted = False 
    AND revision_is_deleted_by_page_deletion = False
    AND array_contains(revision_tags, "contenttranslation")   
    AND wiki_db = '{wiki_db}' 
    AND page_id IN {clean_pageids}
GROUP BY 
    page_id, revision_tags
""".format(**quality_vars))

In [66]:
#https://phabricator.wikimedia.org/T201539
#get list of articles that were created, instep with use of the article translation tool 
at_create = hive.run("""
SELECT
    page_id, 
    revision_tags AS at_create
FROM wmf.mediawiki_history
WHERE
    snapshot = "{MWH_SNAPSHOT}"
    AND event_timestamp >="{contest_start}"
    AND event_timestamp <"{contest_end}"
    AND page_namespace = 0
    AND event_entity = 'page'
    AND event_type = 'create'
    AND revision_is_identity_reverted = False 
    AND revision_is_deleted_by_page_deletion = False
    AND array_contains(revision_tags, "contenttranslation")   
    AND wiki_db = '{wiki_db}' 
    AND page_id IN {clean_pageids}
GROUP BY 
    page_id, revision_tags
""".format(**quality_vars))

In [ ]:
#merge two df above

In [68]:
merge_in_content(at_edits)

In [ ]:
merge_in_content(at_create)

### Relative length

In [11]:
articles = articles.merge(IN_median_vi, on='wikicode', how='left')

In [12]:
articles['relative_page_len'] = articles['page_len']/articles['mpl_index']
articles['relative_page_len'] = articles['relative_page_len'].clip(upper=1)
del articles['mpl_index']

In [13]:
#reorder columns
articles = articles[[
 'wikicode',
 'page_id',
 'page_title',
 'page_len',
 'relative_page_len',
 'revision_tags'
]]

### wikidata Q item

In [14]:
#https://www.mediawiki.org/wiki/Wikibase/Schema/wb_items_per_site
#https://www.mediawiki.org/wiki/Manual:Page_table
#wb_items_per_site site:quarry.wmflabs.org

In [15]:
qid_simple_r = wmf.mariadb.run("""
SELECT
  ips_site_page AS page_title,
  ips_item_id AS QID
FROM  wb_items_per_site  
WHERE ips_site_id = '{wiki_db}' 
AND ips_site_page IN {clean_titles_denormalized}
""".format(**quality_vars), "wikidatawiki")

In [16]:
# change the page_title back to normalized state by replacing spaces with underscores
qid_simple_r['page_title'] = qid_simple_r['page_title'].str.replace(' ', '_')

In [17]:
articles = articles.merge(qid_simple_r[['QID', 'page_title']], on='page_title', how="left").fillna(0)

## Article Creation Date

In [18]:
# DATE_FORMAT(rev_timestamp,"%y-%m-%d") AS first_edited
first_edit_timestamp = wmf.mariadb.run("""
SELECT 
    page_id, 
    rev_timestamp AS first_edited
FROM revision 
JOIN page ON page_id = rev_page
WHERE rev_page IN {clean_pageids}
GROUP BY page_id
""".format(**quality_vars), wiki)

first_edit_timestamp['first_edited']= pd.to_datetime(first_edit_timestamp['first_edited']) 
first_edit_timestamp['first_edited'] = first_edit_timestamp['first_edited'].dt.normalize()

In [19]:
merge_in_content(first_edit_timestamp)

# Pageviews 

In [25]:
pv_1M = hive.run("""
SELECT 
   page_id,
   SUM(view_count) AS views_1M
FROM wmf.pageview_hourly 
WHERE 
  year = 2020
  AND (month >= {contest_end_dt_month} AND day >= {contest_end_dt_day}) 
  AND (month <= {contest_end_dt_1M_month} AND day <= {contest_end_dt_1M_day}) 
  AND agent_type = 'user'
  AND country_code = '{country_code}'
  AND project = '{project}'
  AND page_id IN {clean_pageids}
GROUP BY page_id, view_count
""".format(**quality_vars))

In [26]:
merge_in_content(pv_1M)

### Filter for new articles only

In [27]:
#identify article types based on time of first edit: 'new', 'expanded', or 'post'
articles['article_type'] = articles['first_edited'].apply(create_fill_column)
#drop articles created after the contest ended
articles = articles[articles.article_type !='post']

#create articles_new df only for newly created contest articles
articles_new = first_edit_timestamp[(first_edit_timestamp['first_edited'] >= quality_vars.get('contest_start_dt')) & (first_edit_timestamp['first_edited'] <= quality_vars.get('contest_end_dt'))]
articles_new = articles.merge(articles_new, on=['page_id'], how='inner')

# create a tuple of clean NEW page_ids to query 
clean_new_pageids = tuple(list(articles_new['page_id']))
clean_new_pagetitles_list = list(articles_new['page_title'])
quality_vars.update({'clean_new_pageids': clean_new_pageids})

### Edits & Editors per article for time filtering <a class="anchor" id="editors_active"></a>
[Back to Table of Contents](#toc)

### Edits & editors: all

In [31]:
#https://phabricator.wikimedia.org/T231598
#The (rev_deleted & 4) = 0 condition is to exclude revisions where the user has been RevDeled, as we don't want to leak information about how many such users there are.

edits_editors_all = wmf.mariadb.run("""
SELECT 
    page_id,
    DATE_FORMAT(rev_timestamp,"%y-%m-%d") AS edit_date,    
    revactor_actor
FROM revision_actor_temp
JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
JOIN page ON rev_page = page.page_id
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND (rev_deleted & 4) = 0
    AND rev_page IN {clean_new_pageids}
GROUP BY revactor_rev
""".format(**quality_vars), wiki)

edits_editors_all['edit_date'] = pd.to_datetime(edits_editors_all['edit_date'], format="%y-%m-%d")

DatabaseError: Execution failed on sql '
SELECT 
    page_id,
    DATE_FORMAT(rev_timestamp,"%y-%m-%d") AS edit_date,    
    revactor_actor
FROM revision_actor_temp
JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
JOIN page ON rev_page = page.page_id
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND (rev_deleted & 4) = 0
    AND rev_page IN (())
GROUP BY revactor_rev
': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '))
GROUP BY revactor_rev' at line 11

In [ ]:
# merge with articles_new: first_edit_timestamp df which was filtered for only new articles and put into 'articles_new' df
ee_fe = pd.merge(edits_editors_all, articles[['page_id','page_title', 'first_edited']], on='page_id', how='left').fillna(0)

### Edits & editors: registered, non-bot

In [32]:
edits_editors_reg_r = wmf.mariadb.run("""
SELECT 
    page_id,
    DATE_FORMAT(rev_timestamp,"%y-%m-%d") AS edit_date,    
    revactor_actor
FROM revision_actor_temp
JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
JOIN page ON rev_page = page.page_id
JOIN actor ON (revactor_actor = actor_id)
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND (rev_deleted & 4) = 0
    AND actor_user IS NOT NULL -- user cannot be non-registered
    AND actor_user NOT IN (SELECT ug_user FROM user_groups WHERE ug_group = "bot") -- not a bot
    AND rev_page IN {clean_new_pageids}
GROUP BY revactor_rev
""".format(**quality_vars), wiki)

eern = edits_editors_reg_r.copy()
eern['edit_date'] = pd.to_datetime(eern['edit_date'], format="%y-%m-%d")

DatabaseError: Execution failed on sql '
SELECT 
    page_id,
    DATE_FORMAT(rev_timestamp,"%y-%m-%d") AS edit_date,    
    revactor_actor
FROM revision_actor_temp
JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
JOIN page ON rev_page = page.page_id
JOIN actor ON (revactor_actor = actor_id)
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND (rev_deleted & 4) = 0
    AND actor_user IS NOT NULL -- user cannot be non-registered
    AND actor_user NOT IN (SELECT ug_user FROM user_groups WHERE ug_group = "bot") -- not a bot
    AND rev_page IN ()
GROUP BY revactor_rev
': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ')
GROUP BY revactor_rev' at line 14

In [ ]:
# merge with articles_new: first_edit_timestamp df which was filtered for only new articles and put into 'articles_new' df
eern_fe = pd.merge(eern, articles_new[['page_id','page_title', 'first_edited']], on='page_id', how='left').fillna(0)

In [ ]:
#calculate total edits
eern_fe['total_edits'] = eern_fe.groupby(['page_id', 'page_title'])['edit_date'].agg('count').reset_index(name='edits_1M')

#### filter for edits by time period and use those in groupby counts below

In [ ]:
# create a timedelta column
eern_fe['edit_td'] = eern_fe['edit_date']-eern_fe['first_edited']

In [ ]:
m1['revactor_actor'].nunique()

In [ ]:
# filter for only edits in first 30 days
m1 = eern_fe[eern_fe['edit_td'] <= pd.Timedelta(30, unit='d')]

# filter for only edits in first 60 days
m2 = eern_fe[eern_fe['edit_td'] <= pd.Timedelta(60, unit='d')]

#### editor counts per article at 1M, 2M

In [ ]:
editors_M1_counts = m1.groupby(['page_id', 'page_title'])['revactor_actor'].nunique().reset_index(name='editors_1stM')

In [ ]:
editors_M2_counts = m2.groupby(['page_id', 'page_title'])['revactor_actor'].nunique().reset_index(name='editors_2ndM')

In [ ]:
editors_by_M_calculations = pd.merge(editors_M1_counts, editors_M2_counts[['page_id', 'editors_2ndM']],
                               on='page_id', 
                               how='left').fillna(0)

#### edit counts per article

In [ ]:
edit_counts_1M = m1.groupby(['page_id', 'page_title'])['edit_date'].agg('count').reset_index(name='edits_1M')

In [ ]:
edit_counts_2M = m2.groupby(['page_id', 'page_title'])['edit_date'].agg('count').reset_index(name='edits_2M')

In [ ]:
edits_by_M_calculations = pd.merge(edit_counts_1M, edit_counts_2M[['page_id', 'edits_2M']],
                               on='page_id', 
                               how='left').fillna(0)

In [ ]:
articles = pd.merge(articles, editors_M1_counts[['page_id', 'editors_1stM']],
                               on='page_id', 
                               how='left').merge(edit_counts_1M[['page_id', 'edits_1M']], 
                               on='page_id', 
                               how='left').fillna(0)

### Edits & timestamps

In [33]:
# This timestamp is updated whenever the page changes in a way requiring it to be re-rendered, invalidating caches. 
#Aside from editing, this includes permission changes, creation or deletion of linked pages, and alteration of contained templates. 
#[[mw:Manual:Revision_table]] and [[mw:Manual:Page_table]]. Only show latest edits does an inner join from revision table to page table on rev_id = page_latest .
#https://www.mediawiki.org/wiki/Manual:Revision_table
#https://github.com/x-tools/xtools/blob/master/src/AppBundle/Repository/ArticleInfoRepository.php#L162-L171
#https://xtools.wmflabs.org/articleinfo/pa.wikipedia.org/ਏਸ਼ੀਆ

edits = wmf.mariadb.run("""
SELECT 
    rev_page AS page_id, 
    COUNT(rev_id) AS num_edits_all_time,
    SUM(rev_minor_edit) AS minor_edits_all_time
FROM revision
JOIN page ON page_id = rev_page
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND rev_page IN {clean_new_pageids}
GROUP BY rev_page
""".format(**quality_vars), wiki)

timestamps = wmf.mariadb.run("""
SELECT 
    rev_page AS page_id, 
    max(rev_timestamp) AS last_edited 
FROM revision 
JOIN page 
  ON page_id = rev_page
WHERE rev_page IN {clean_new_pageids}
AND rev_id = page_latest
GROUP BY rev_page
""".format(**quality_vars), wiki)

DatabaseError: Execution failed on sql '
SELECT 
    rev_page AS page_id, 
    COUNT(rev_id) AS num_edits_all_time,
    SUM(rev_minor_edit) AS minor_edits_all_time
FROM revision
JOIN page ON page_id = rev_page
WHERE rev_page = page_id
    AND rev_timestamp > 0 
    AND rev_page IN ()
GROUP BY rev_page
': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ')
GROUP BY rev_page' at line 9

In [ ]:
timestamps['last_edited']= pd.to_datetime(timestamps['last_edited']) 
timestamps['last_edited'] = timestamps['last_edited'].dt.normalize()

merge_in_content(edits)
merge_in_content(timestamps)

### Editors

In [ ]:
# editors: total number of unique editors, including IP editors and bots...all editors of all edits, microcontributions
#https://www.mediawiki.org/wiki/Manual:Page_table
#https://www.mediawiki.org/wiki/Manual:Revision_actor_temp_table
#https://www.mediawiki.org/wiki/Manual:Revision_table
#https://www.mediawiki.org/wiki/Help:RevisionDelete
#adapted from https://phabricator.wikimedia.org/T231598#5465711
#questioned in https://phabricator.wikimedia.org/T234560#5545319
#taken into account: rev_deleted to avoid leaking information re: how many distinct users were involved in revision-deleted edits
tuepa_r = wmf.mariadb.run("""
SELECT 
    page.page_id AS page_id,
    COUNT(DISTINCT revactor_actor) AS all_editors_of_all_edits
FROM revision_actor_temp
  JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
  JOIN page ON rev_page = page.page_id  
WHERE rev_page = page.page_id 
  AND (rev_deleted & 4) = 0
  AND page.page_id IN {clean_new_pageids}
GROUP BY page_id
""".format(**quality_vars), wiki)

In [ ]:
# query post Morten review
#editors: total, unique, non-bot, registered editors that made non-minor edits
tunbre_r = wmf.mariadb.run("""
SELECT 
  revision.rev_page AS page_id,
  COUNT(DISTINCT revactor_actor) AS editors_nm
FROM revision_actor_temp
  JOIN revision ON (revactor_rev = rev_id)
  JOIN actor ON (revactor_actor = actor_id)
WHERE (rev_deleted & 4) = 0
  AND rev_minor_edit = 0
  AND actor_user IS NOT NULL -- user cannot be non-registered
  AND actor_user NOT IN (SELECT ug_user FROM user_groups WHERE ug_group = "bot") -- not a bot
  AND revision.rev_page IN {clean_new_pageids}
GROUP BY revision.rev_page
""".format(**quality_vars), wiki)

In [ ]:
# editors: total number of unique editors, including IP editors and bots...all editors of all edits, microcontributions
#https://www.mediawiki.org/wiki/Manual:Page_table
#https://www.mediawiki.org/wiki/Manual:Revision_actor_temp_table
#https://www.mediawiki.org/wiki/Manual:Revision_table
#https://www.mediawiki.org/wiki/Help:RevisionDelete
#adapted from https://phabricator.wikimedia.org/T231598#5465711
#questioned in https://phabricator.wikimedia.org/T234560#5545319
#taken into account: rev_deleted to avoid leaking information re: how many distinct users were involved in revision-deleted edits
tueme_r = wmf.mariadb.run("""
SELECT 
    page.page_id,
    COUNT(DISTINCT revactor_actor) AS micro_editors
FROM revision_actor_temp
  JOIN revision ON(revactor_rev = rev_id AND revactor_page = rev_page)
  JOIN page ON rev_page = page.page_id  
WHERE rev_page = page.page_id 
  AND (rev_deleted & 4) = 0
  AND page.page_id IN {clean_new_pageids}
  AND rev_minor_edit = 1
GROUP BY page_id
""".format(**quality_vars), wiki)

In [ ]:
# updated!
#total unique IP editors, 
#see also: https://phabricator.wikimedia.org/T231605
#https://meta.wikimedia.org/wiki/IP_Editing:_Privacy_Enhancement_and_Abuse_Mitigation/Research
#https://github.com/nettrom/AHT-block-effectiveness-2018
#https://github.com/wikimedia-research/AHT-IP-edits-2019/blob/master/edit_usefulness.ipynb
#https://meta.wikimedia.org/wiki/User:Benjamin_Mako_Hill/Research_on_the_value_of_IP_Editing
tuipe_r = wmf.mariadb.run("""
SELECT 
  revision.rev_page AS page_id,
  COUNT(DISTINCT revactor_actor) AS IP_editors
FROM revision
  JOIN revision_actor_temp ON (rev_id = revactor_rev)
  JOIN actor ON (revactor_actor = actor_id)
WHERE (rev_deleted & 4) = 0
  AND actor_user IS NULL -- non-registered user
  AND revision.rev_page IN {clean_new_pageids}
GROUP BY rev_page
""".format(**quality_vars), wiki)

#### editor calculations

In [ ]:
editor_calculations = pd.merge(tuepa_r, tunbre_r[['page_id', 'editors_nm']], on='page_id', how='left').fillna(0)

In [ ]:
editor_calculations = pd.merge(editor_calculations, tuipe_r[['page_id', 'IP_editors']],
                               on='page_id', 
                               how='left').merge(tueme_r[['page_id', 'micro_editors']], 
                               on='page_id', 
                               how='left').fillna(0)

In [ ]:
merge_in_content(editor_calculations)

### Article's talk page activity

In [ ]:
# https://meta.wikimedia.org/wiki/Research:Usage_of_talk_pages/2019-11-11#arwiki
#https://meta.wikimedia.org/wiki/Research:Newsletter/2011/August
#http://jodischneider.com/pubs/sac2011.pdf
#https://meta.wikimedia.org/wiki/Research:Newsletter/2015/May#Editors_who_use_user_talk_pages_are_more_involved_in_high-quality_articles
#https://meta.wikimedia.org/wiki/Research:New     vI will be happy to take a look at your queries. I will be happy to take a look at your queries. vsletter/2017/May#cite_note-9
#https://www.opensym.org/wp-content/uploads/2018/07/OpenSym2018_paper_14.pdf
#https://phabricator.wikimedia.org/T214935 -- on talk page click through rates
#SQL:
#https://github.com/wikimedia-research/Talkcicity/blob/master/retrieve_talkpage_data.R
#https://github.com/x-tools/xtools/blob/master/src/AppBundle/Repository/ArticleInfoRepository.php#L221-L226
#https://github.com/wikimedia-research/2019-10-talk-pages-baseline-metrics/blob/master/2019-10-talk-page-contributors-analysis.ipynb


In [ ]:
talk_page_edits = wmf.mariadb.run(""" 
SELECT 
    pa.page_id, 
    SUM(IF(rev_id IS NOT NULL, 1, 0)) AS talk_page_edits
FROM page pa
LEFT JOIN page pt
    ON pa.page_title = pt.page_title
    AND pt.page_namespace = 1
LEFT JOIN revision
    ON pt.page_id = rev_page
WHERE pa.page_id IN {clean_new_pageids}
    AND pt.page_namespace = 1
    AND (rev_deleted & 4) = 0
GROUP BY pa.page_id
""".format(**quality_vars), wiki)

In [ ]:
merge_in_content(talk_page_edits)

### Watchlist

In [ ]:
# https://phabricator.wikimedia.org/source/mediawiki/browse/master/maintenance/tables.sql

article_watch_count = wmf.mariadb.run("""
SELECT page_id,
       COUNT(*) AS watch_count
FROM watchlist
JOIN page 
     ON (wl_title = page_title AND wl_namespace = page_namespace)
WHERE page_namespace = 0 
    AND page_id IN {clean_new_pageids}
GROUP BY page_id
""".format(**quality_vars), wiki)

In [ ]:
merge_in_content(article_watch_count)

### Revert rate

In [ ]:
# see the revert rate (rr) notebook

In [ ]:
rr_replicas = wmf.mariadb.run("""
SELECT page_id, 
       COUNT(DISTINCT rev_id) AS revertrate
FROM revision
 JOIN change_tag 
     ON ct_rev_id = rev_id
 JOIN change_tag_def 
     ON ct_id = ctd_id
 JOIN page 
     ON rev_page = page_id
WHERE ctd_name IN ('mw-rollback', 'mw-undo')
    AND page_id IN {clean_new_pageids}
GROUP BY page_id
""".format(**quality_vars), wiki)

In [ ]:
merge_in_content(rr_replicas)

# links <a class="anchor" id="links"></a>
[Back to Table of Contents](#toc)

In [ ]:
# pagelinks and redirects examples
#https://github.com/nettrom/suggestbot/blob/master/tool-labs/link-rec/inlink-table-updater.py#L227

#linking
#https://en.wikipedia.org/wiki/Special:WhatLinksHere/Wikipedia:Manual_of_Style/Linking
#https://en.wikipedia.org/wiki/Wikipedia:Manual_of_Style/Linking#General_principles
#https://www.mediawiki.org/wiki/API:Links

#backlinks+
#https://dispenser.info.tm/~dispenser/cgi-bin/backlinkscount.py (backlinks)
#https://github.com/wikimedia/mediawiki-api-demos/blob/master/python/get_backlinks.py

#tables
#pagelinks contains links to other pages on the same wiki...provide cohesion and utility
#https://www.mediawiki.org/wiki/Manual:Pagelinks_table (internal links in the same wiki, from the page)

#externallinks contains links to elsewhere, outside of all wikis
#https://www.mediawiki.org/wiki/Manual:Externallinks_table (external links, from the page)

#interwikilinks links an article in one language to the same article in another language. For most articles these are stored on Wikidata. 
#https://en.wikipedia.org/wiki/Help:Interwiki_linking
#https://www.mediawiki.org/wiki/Manual:Iwlinks_table

#langlinks links that point to a page on another wiki (e.g. [[mw:Product Analytics]] links to the PA team’s page on MediaWiki-wiki.
#https://en.wikipedia.org/wiki/Help:Interlanguage_links#Local_links
#https://www.mediawiki.org/wiki/Manual:Langlinks_table

### pagelinks <a class="anchor" id="pagelinks"></a>
[Back to Table of Contents](#toc)

##### TECHNICAL NOTE FROM MORTEN: `pl.pl_namespace = 0` is not needed in the `WHERE` clause because you're operating on page IDs. Unless you also want to specificy that only articles are to be included, though. But I would in that case restrict that much earlier, when converting page titles to page IDs.

In [ ]:
# pagelinks: linking to articles within the same wiki
opl_r = wmf.mariadb.run("""
    SELECT 
        pl.pl_from, 
        link.page_id AS plpage,
        link.page_title AS plpage_title,
        redir.page_id AS rpage,
        redir.page_title AS rpage_title,
        redir.page_is_redirect AS is_double_redirect
    FROM pagelinks AS pl
    JOIN page AS link
        ON (pl.pl_namespace=link.page_namespace
        AND pl.pl_title=link.page_title)
    LEFT JOIN redirect AS rd
        ON link.page_id=rd.rd_from
    LEFT JOIN page AS redir
        ON (rd.rd_namespace=redir.page_namespace
        AND rd.rd_title=redir.page_title)
    WHERE pl.pl_namespace=0 AND pl.pl_from IN {clean_new_pageids}
""".format(**quality_vars), wiki)

In [ ]:
# check to see if any of the page_ids are double redirects
((opl_r['is_double_redirect']==1).any())

In [ ]:
# check to see if an anchor's target is duplicated...for duplicate instances of a link within a single page
#pagelinks_r[(pagelinks_r.duplicated('pl_from') & pagelinks_r.duplicated('lpage'))] #checks for duplicates in either column
opl_r[opl_r.duplicated(['pl_from','plpage'])] #checks for duplicates in two columns at the same time

In [ ]:
# because there are no duplicated targets from each anchor, we can count the number of occurrences for each anchor as the target_count
opls = opl_r['pl_from'].value_counts().to_frame().reset_index().rename(columns={'index':'page_id', 'pl_from':'oplinks'})

In [ ]:
merge_in_content(opls)

### external links <a class="anchor" id="extlinks"></a>
[Back to Table of Contents](#toc)

In [ ]:
# externallinks 
#https://www.mediawiki.org/wiki/Manual:Externallinks_table
oel_r = wmf.mariadb.run("""
SELECT el_from, 
       el_to
FROM externallinks AS el
WHERE el.el_from IN {clean_new_pageids}
""".format(**quality_vars), wiki)

In [ ]:
# check to see if any of the targets are duplicated in the article
oel_r.duplicated()

In [ ]:
# if no external links are duplicated, then count targets arising from each anchor:
#because there are no duplicated targets from each anchor, we can count the number of occurrences for each anchor as the target_count
oextlinks = oel_r['el_from'].value_counts().to_frame().reset_index().rename(columns={'index':'page_id', 'el_from':'oelinks'})

In [ ]:
merge_in_content(oextlinks)

#### Categorize opls & oextlinks

In [ ]:
# use bins to seperate number of pagelinks into categories
#seperate number of links into bins defined for oel_bins 
opls["binned_links"] = pd.cut(opls["oplinks"], oel_bins, right=False)

#sort
opls["binned_links"].value_counts(sort = False)

In [ ]:
# use bins to seperate number of pagelinks into categories
#seperate number of links into bins defined for oext_bins 
oextlinks["binned_links"] = pd.cut(oextlinks["oelinks"], oext_bins, right=False)

#sort
oextlinks["binned_links"].value_counts(sort = False)

### Incoming pagelinks <a class="anchor" id="inpagelinks"></a>
[Back to Table of Contents](#toc)

### NOTES FROM MORTEN:
I'd steal the backlinks query from here: https://github.com/nettrom/suggestbot/blob/master/tool-labs/link-rec/inlink-table-updater.py#L229 It already uses a list of page IDs as the basis for the query, but does limit links to within the article namespace.

Then I'd just remove `AS ilc_page_id` and change `AS ilc_numlinks` to `AS numlinks` or something. That query is pretty optimized, and also counts inlinks coming in through redirects. You'd definitely want to do something like this using the replicated MW databases than going through any API. If you're working with large numbers of articles (e.g. thousands), I can show you how to iterate over them in groups to speed things up, by the way.

In [ ]:
# incoming pagelinks: links from within the same wiki
#https://www.mediawiki.org/wiki/Manual:Pagelinks_table
#resource: https://github.com/nettrom/suggestbot/blob/master/tool-labs/link-rec/inlink-table-updater.py#L229
#	pl_from, pl_from_namespace #anchor
#	pl_namespace	pl_title #target
incoming_pagelinks_r = wmf.mariadb.run("""
    SELECT 
        link.page_id AS page_id,
        pl.pl_title AS page_title,
        pl.pl_from AS in_pagelinks
    FROM pagelinks AS pl
    JOIN page AS link
        ON (pl.pl_namespace=link.page_namespace
        AND pl.pl_title=link.page_title)
    LEFT JOIN redirect AS rd
        ON link.page_id=rd.rd_from
    LEFT JOIN page AS redir
        ON (rd.rd_namespace=redir.page_namespace
        AND rd.rd_title=redir.page_title)
    WHERE pl.pl_namespace=0
        AND link.page_id IN {clean_new_pageids}
""".format(**quality_vars), wiki)

In [ ]:
incoming_pagelinks = incoming_pagelinks_r['page_id'].value_counts().to_frame().reset_index().rename(columns={'index':'page_id', 'page_id':'ipl_count'})

In [ ]:
merge_in_content(incoming_pagelinks)

# iwsitelinks

##### create qid list from column for pulling categories (use quid_simple_r from above)

In [ ]:
qid_simple_r.rename(columns={'page_title':'title','q_id':'QID'},inplace=True)

In [ ]:
qids = tuple(list(qid_simple_r['QID']))

In [ ]:
quality_vars.update({'qids': qids})

In [ ]:
#https://www.wikidata.org/wiki/Help:Sitelinks
#https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q42&props=sitelinks
        
ips_sites = wmf.mariadb.run("""
SELECT
  linked_item.ips_item_id AS QID,
  GROUP_CONCAT(ips_site_id SEPARATOR ', ') AS iwsites,
  COUNT(ips_site_page) AS iwsitelinks
FROM (
      SELECT ips_item_id
      FROM wb_items_per_site
      WHERE ips_site_id = '{wiki_db}' AND ips_item_id IN {qids}
    ) AS linked_item
LEFT JOIN wb_items_per_site 
  ON linked_item.ips_item_id = wb_items_per_site.ips_item_id
LEFT JOIN page 
  ON linked_item.ips_item_id = page.page_id
GROUP BY page_id
""".format(**quality_vars), "wikidatawiki")

In [ ]:
articles = articles.merge(ips_sites[['iwsitelinks', 'iwsites', 'QID']], on='QID', how="left").fillna(0)

## Save data

In [ ]:
# articles.to_csv("./data/raw/query_results/content_quality/{}.csv", sep=',', encoding = 'utf-8')

# Assess collected data